In [2]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import random
import numpy as np

# Read the Data

In [3]:
# read data
monday=pd.read_csv('monday.csv', sep=';')
tuesday=pd.read_csv('tuesday.csv', sep=';')
wednesday=pd.read_csv('wednesday.csv', sep=';')
thursday=pd.read_csv('thursday.csv', sep=';')
friday=pd.read_csv('friday.csv', sep=';')


# Define a class for repetative pre-processing

In [4]:
class dframe:
    def df(day, name):
        day['timestamp'] = pd.to_datetime(day['timestamp'])
        day.sort_values(by=['customer_no', 'timestamp'], ascending=True, inplace=True)
        day['customer_id'] = name + day['customer_no'].astype(str)
        day.set_index('timestamp', inplace=True)
        day = day.groupby('customer_no').resample('1T').first()
        day.fillna(method="ffill", inplace=True)
        day.reset_index(drop=True, inplace=True)
        day['state_shift']=day['location'].shift(-1)
        day.rename(columns={'location': 'before', 'state_shift': 'after'}, inplace=True)
        day['after'].loc[(day['before'] == 'checkout')] = 'checkout'

        return day       

# Make an instance to generate data frames and concatenation

In [5]:
resampled = dframe
monday = resampled.df(monday, 'm')
tuesday = resampled.df(tuesday, 'tu')
wednesday = resampled.df(wednesday, 'w')
thursday = resampled.df(thursday, 'th')
friday = resampled.df(friday, 'f')

/Users/Disalo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [6]:
frames=[monday, tuesday , wednesday , thursday ,friday]
df = pd.concat(frames, ignore_index=True)
print(df.shape)
df.head()

(53903, 4)


,customer_no,before,customer_id,after
0,1.0,dairy,m1,dairy
1,1.0,dairy,m1,checkout
2,1.0,checkout,m1,checkout
3,2.0,dairy,m2,dairy
4,2.0,dairy,m2,dairy


# Declare possible states in a list

In [7]:
possible_states = ['checkout', 'dairy', 'drinks', 'fruit', 'spices']

# Extract transition matrix

In [8]:
transition_matrix=pd.crosstab(df['before'],df['after'],normalize=0)

In [9]:
transition_matrix

after,checkout,dairy,drinks,fruit,spices
before,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.102684,0.737129,0.058628,0.050076,0.051483
drinks,0.215505,0.010899,0.598499,0.088012,0.087086
fruit,0.201070,0.095931,0.055166,0.597151,0.050681
spices,0.149912,0.193564,0.163135,0.091126,0.402262


# Save transition matrix as .CSV file

In [10]:
transition_matrix.to_csv('transition_matrix.csv')